In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from copy import deepcopy
import sys
import datasets
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('../experiments/')
import os
import scipy
import pandas as pd
import numpy as np
import transformers
import sys
from os.path import join
import datasets
from dict_hash import sha256
import numpy as np
from torch.autograd import grad
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
import imodelsx.llm
import word_distr
import re
import warnings

In [ ]:
checkpoint = 'gpt2-xl'
prefix = 'My favorite ice cream flavor is'
model = imodelsx.llm.load_hf_model(checkpoint)
tokenizer = imodelsx.llm.load_tokenizer(checkpoint)
d = word_distr.get_next_word_distr(
    prefix,
    model,
    tokenizer,
    num_tokens=2,
    cdf_threshold=0.7,
    batch_size=2048,
)
d

### Brute force search everything (just to check)

In [ ]:
prefix_embs = tokenizer.encode(prefix, return_tensors="pt")[0]
input_ids_tok1 = [
    torch.concatenate(
        (prefix_embs, torch.LongTensor([i])))
    for i in range(voc_size)
]
input_ids_tok1 = torch.stack(input_ids_tok1)

In [ ]:
bigram_logits = np.zeros((voc_size, voc_size), dtype=np.double)
unigram_logits = get_unigram_logits(prefix, tokenizer, model)
for i in tqdm(range(0, voc_size, batch_size)):
    input_ids = input_ids_tok1[i:i + batch_size]
    bigram_logits[i:i + batch_size] = \
        model(input_ids).logits[:, -1, :].detach().cpu().numpy()

In [ ]:
# normalize
unigram1_probs = scipy.special.softmax(unigram_logits).flatten()
unigram2_probs = scipy.special.softmax(bigram_logits, axis=1)
assert np.allclose(unigram1_probs.sum(), 1)

In [ ]:
# multiply each row of unigram2_probs by the corresponding value in unigram_probs
bigram_probs = unigram2_probs * unigram1_probs[:, None]
assert np.allclose(bigram_probs.sum(axis=1), unigram1_probs, atol=1e-6)
assert np.allclose(bigram_probs.sum(), 1, atol=1e-6)
print('shapes', bigram_probs.shape)

In [ ]:
np.cumsum(unigram1_probs.astype(np.float32))

In [ ]:
bigram_probs.shape

In [ ]:
unigram1_probs.shape

In [ ]:
unigram1_probs.sum()

In [ ]:
bigram_probs.sum()

In [ ]:
(bigram_probs.sum(axis=1) - unigram1_probs).min()

In [ ]:
# find top 100 bigrams (row, col) pairs - this took like 20 mins
bigram_probs_flat = bigram_probs.flatten()
bigram_probs_flat_sorted = np.sort(bigram_probs_flat)[::-1]
bigram_probs_flat_sorted_idx = np.argsort(bigram_probs_flat)[::-1]
bigram_probs_sorted = bigram_probs_flat_sorted.reshape(bigram_probs.shape)
bigram_probs_sorted_idx = np.unravel_index(
    bigram_probs_flat_sorted_idx, bigram_probs.shape)
bigram_probs_sorted_idx = np.stack(bigram_probs_sorted_idx, axis=1)
bigram_probs_sorted_idx = bigram_probs_sorted_idx[:100]

In [ ]:
top_bigrams = tokenizer.batch_decode(bigram_probs_sorted_idx)